# 201710957 Comp. sci. 이유진

## week5


### 연습문제 1

다음 4건의 데이터에 대해 Merkle Root 값을 계산하고 출력하세요.

중간 노드의 AB, CD의 해시도 출력하세요.

* txA = 'Hello'

* txB = 'How are you?'

* txC = 'This is Thursday'

* txD = 'Happy new Year'



In [38]:
import hashlib

txA = 'Hello'.encode()
txB = 'How are you?'.encode()
txC = 'This is Thursday'.encode()
txD = 'Happy new Year'.encode()


hashA=hashlib.sha256(txA)
hashB=hashlib.sha256(txB)
hashC=hashlib.sha256(txC)
hashD=hashlib.sha256(txD)

hashAswap="".join(reversed([hashA.hexdigest()[i:i+2] for i in range(0, hashA.digest_size*2, 2)]))
hashBswap="".join(reversed([hashB.hexdigest()[i:i+2] for i in range(0, hashB.digest_size*2, 2)]))
hashAB=hashAswap+hashBswap
doubleHashAB=hashlib.sha256(hashAB.encode())
print("node AB의 hash 값 : "+doubleHashAB.hexdigest())

hashCswap="".join(reversed([hashC.hexdigest()[i:i+2] for i in range(0, hashC.digest_size*2, 2)]))
hashDswap="".join(reversed([hashD.hexdigest()[i:i+2] for i in range(0, hashD.digest_size*2, 2)]))
hashCD=hashCswap+hashDswap
doubleHashCD=hashlib.sha256(hashCD.encode())
print("node CD의 hash 값 : "+doubleHashCD.hexdigest())

hashABswap="".join(reversed([doubleHashAB.hexdigest()[i:i+2] for i in range(0, doubleHashAB.digest_size*2, 2)]))
hashCDswap="".join(reversed([doubleHashCD.hexdigest()[i:i+2] for i in range(0, doubleHashCD.digest_size*2, 2)]))
hashABCD = hashABswap + hashCDswap
doubleHashABCD=hashlib.sha256(hashABCD.encode()).hexdigest()

print("   Merkle Root 값 : "+doubleHashABCD)

node AB의 hash 값 : a7dca36b1e6fe56e1d7bc34fe56ceea8ab64f47a55e929b712fab58885dcdc37
node CD의 hash 값 : 15524f5c1b536c2fe0f717fd8b1bbede37f94654fae6f47c20521ce476c8e0b2
   Merkle Root 값 : 00aa61af726d4cd0d5a5e0d2e3e3280552acb6d587c15413a713b7a96275cefc



### 연습문제 2

친구에게  주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.

안되면 친구의 주소를 만들고 전송한다.

잔고의 증가분을 출력하세요.

소요된 gas비용 출력하세요.



In [1]:
!geth --exec "eth.getBalance(eth.accounts[0])" attach http://localhost:8445

350000000000000000000


In [2]:
!geth --exec "eth.sendTransaction({from:eth.accounts[0], to:eth.accounts[1] ,value:10000})" attach http://localhost:8445 

"0xba4c73c94f3461e84dc7cbd53b81e19a6a1917163b6b6f611b69ba90ee541e57"


In [3]:
!geth --exec "eth.getBalance(eth.accounts[0])" attach http://localhost:8445

350000000000000000000


In [33]:
%%writefile e_testTran.js
miner.setEtherbase(eth.accounts[0]);
console.log('coinbase: ', eth.coinbase);
var bal1=eth.getBalance(eth.coinbase);
var bal2=eth.getBalance(eth.accounts[1]);


console.log('\nsender balance in ether: ', web3.fromWei(bal1,"ether"));
var wei2 = web3.fromWei(bal2,"ether")
console.log('receiver balance in ether: ', web3.fromWei(bal2,"ether"));

console.log('median gas price: ', eth.gasPrice);
console.log('block number: ', eth.blockNumber);
console.log('tranaction count: ', eth.getTransactionCount(eth.coinbase));
eth.sendTransaction({from:eth.coinbase, to:eth.accounts[1],value:10000});



var bal1new=eth.getBalance(eth.coinbase);
var bal2new=eth.getBalance(eth.accounts[1]);
console.log('- new sender balance in ether: ', web3.fromWei(bal1new,"ether"));

var wei2new =  web3.fromWei(bal2new,"ether")
console.log('- new receiver balance in ether: ', web3.fromWei(bal2new,"ether"));
console.log('- block number: ', eth.blockNumber);
console.log('- tranaction count: ', eth.getTransactionCount(eth.coinbase))


console.log('\n2-1 : ' + ( wei2new - wei2 ) )
console.log('2-2 : ' + eth.gasPrice )
console.log()

Overwriting e_testTran.js


In [1]:
!geth --exec "loadScript('e_testTran.js')" attach http://localhost:8445

coinbase:  0x478e5deacb2c8b60452f7d9822e50643bb78c1d3

sender balance in ether:  350
receiver balance in ether:  130
median gas price:  1000000000
block number:  125
tranaction count:  0
- new sender balance in ether:  350
- new receiver balance in ether:  130
- block number:  125
- tranaction count:  0

2-1 : 0
2-2 : 1000000000

true


잔고의 증가분(2-1) 은 0 이다. 송금을 했음에도 불구하고 잔고가 증가하지않았다.

### 연습문제 3

해시는 100미만의 양수로 정해진다고 하자.

NONCE는 반복회수로만 쓰이고 무작위 수를 생성하는데 입력되지는 않는다.

목표해시는 난이도에 따라 결정이 된다고 하자.

마이닝을 해서, 목표 해시를 찾아보자.

- 90을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력

- 10을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력



In [53]:
import hashlib
count = 0
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    
    if guessHash[:2]=='90':
        found=True
        
    NONCE+=1
  
    
    
print(guessHash)
print(NONCE)
print("---------------------------")

NONCE = 0
found = False
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:2]=='10':
        found=True
    NONCE+=1
    
  

print(guessHash)
print(NONCE)

90a2efc023e8445d7272b5fe2b4902deb9f789badec4e83e33f815826eaf3486
23
---------------------------
103cd851bf325328103e7d92b02811ddc103c9114c2084cd21efd1fb9767b796
188
